# Imports, Functions & Testing

In [1]:
import pandas as pd
import numpy as np
import requests
import re
import pickle
from bs4 import BeautifulSoup

In [2]:
def gen_election_cleaner(df, verbose=1):
    import pandas as pd
    
    df_cln = df.copy()
    
    if verbose in [1,2]:
        ## Printing startpoint
        print('O.G. Copy:')
        print('---'*20)
        display(df_cln)

    ## Setting columns manually + Q.C.
    df_cln.columns = ['Null', 'Party', 'Candidate', 'Votes', '%of_Vote']
    if verbose == 2:
        print('\nNew Column Headers:')
        print('---'*20)
        display(df_cln)

    ## Dropping irrelevant information + Q.C.
    df_cln.drop(index=[0, 6, 8], inplace=True)
    if verbose == 2:
        print('\nDropping Rows:')
        print('---'*20)
        display(df_cln)

    ## Flip and place Turnout as its own feature + Q.C.
    for idx, v, k in zip(df_cln.index, df_cln['Null'], df_cln['Null'].isna()):
        if not k:
            df_cln[v] = df_cln['Party'][idx]

    ## Removing the row with turnout info/column with nulls + Q.C.
    df_cln.drop(columns='Null', index=7, inplace=True)
    if verbose in [1,2]:
        print('\nFinal Version:')
        print('---'*20)
        display(df_cln)

    return df_cln

In [3]:
def wiki_senate_scraper():
    import pandas as pd
    import numpy as np
    import requests
    from bs4 import BeautifulSoup
    
    ## Starting page + Q.C. of response
    start_url = 'https://en.wikipedia.org/wiki/List_of_United_States_Senate_elections'
    start_resp = requests.get(start_url)
    print(f'Starting Response: {start_resp}')
    
    ## Creating soup + pulling all links for senate elections after 17th amendment
    start_soup = BeautifulSoup(start_resp.text, 'html.parser')
    start_links = start_soup.findAll('a')
    start_sen_links = start_links[78:134] ## Previously located
    
    ## Base of url for all senate election pages
    base_url = 'https://en.wikipedia.org'
    
    ## Loop same process for all links + storage
    yr_dfs = {}
    yr_tocs = {}
    count = 0
    for link in start_sen_links:

        ## Q.C during execute
        count += 1
        if count in [25, 50]:
            print('Checkpoint! (25 loops)')
        
        ## Collecting strings for use
        end_url = link.get('href')
        year = link.get_text()
        full_url = base_url + end_url
        
        ## Making soup + collecting all tables
        link_resp = requests.get(full_url)
        link_soup = BeautifulSoup(link_resp.text, 'html.parser')
        link_tables = link_soup.findAll('table', attrs={'class': ['wikitable', 'infobox vevent', 'infobox']})
        
        ## Collecting list of states with elections in each year
        link_toc = link_soup.find('div', attrs={'id':'toc'})
        link_toc = link_toc.findAll('a', href=is_state)
        toc_list = [tag.get('href').replace('#', '') for tag in link_toc]
        
        ## Converting to dataframe + storage
        elect_df = pd.read_html(str(link_tables))
        yr_dfs[year] = elect_df
        yr_tocs[year] = toc_list
    
    print(f'Total pages scraped: {count}')
    return yr_dfs, yr_tocs

## https://www.crummy.com/software/BeautifulSoup/bs4/doc/#kinds-of-filters

In [4]:
def is_state(href):
    states_list = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 
                   'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
                   'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
                   'Kentucky', 'Louisiana', 'Maine', 'Maryland',
                   'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
                   'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New_Hampshire',
                   'New_Jersey', 'New_Mexico', 'New_York', 'North_Carolina',
                   'North_Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
                   'Rhode_Island', 'South_Carolina', 'South_Dakota',
                   'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia',
                   'Washington', 'West_Virginia', 'Wisconsin', 'Wyoming']
    for state in states_list:
        if state in href:
            return href

In [5]:
def election_collector(dict_tables, dict_lists):
    
    ## Requirement of proper dictionaries
    if dict_tables.keys() != dict_lists.keys():
        print('**WARNING**')
        print('Keys do not match in dictionaries passed. Adjust and try again.')
        return '***'*10
    
    ## Containers for results
    coll_elects = {}
    
    ## List creation for looping through dicts
    yr_list = list(dict_lists.keys())
    
    ## Looping + storage
    for year in yr_list:
        print('***'*10)
        print(f"Collecting {year}'s elections...")
        yr_tables = dict_tables[year]
        yr_toc = dict_lists[year]
        
        ## Creating containers to further separate data
        yr_sum_ldrs = yr_tables[2]
        most_tables = yr_tables[3:]
        yr_summary = []
        yr_states = []
        count = 0
        
        for i, df in enumerate(most_tables):
            if count < 1:
                if df.shape[1] is 6 and most_tables[i+1].shape[1] is 6:
                    count += 1
                    yr_summary.append(most_tables[i])
                    yr_summary.append(most_tables[i+1])
                    continue
                elif df.shape[1] is 6:
                    count += 1
                    yr_summary.append(most_tables[i])
            
            if df.shape[1] in [4,5,6] and df.shape[0] < 15:
                yr_states.append(most_tables[i])
        
        coll_elects[year] = [yr_sum_ldrs, yr_summary, yr_states]
        
    print('\n')
    print('---'*20)
    print(f'Collection Complete: Data is for {yr_list[0]} - {yr_list[-1]}')
    print('---'*20)
    return coll_elects

In [12]:
def yr_summary_collector(election_collection):
    
    ## Container for results
    yr_summaries = {}
    
    ## Extraction as a list for iteration
    for year in election_collection:
        elect_list = [election_collection[year]]
        
        ## Looping through year's collection
        for coll in elect_list:
            ## Container for mid-results
            holder = []
            ## Looping through each table in 'year summary' group
            for table in coll[1]:
                ## Variable to filter unwanted tables out
                has_elected = False
                ## Looping through each row in 'results' column (idx 4)
                for u in table.iloc[:,4]:
                ## Checking for str to 'flag' table as collectable
                    if isinstance(u, str) and 'elected' in u:
                        has_elected = True
                if has_elected == True:
                    holder.append(table)
            ## Check if multiple tables in 'year summary' group
            if len(holder) > 1:
                is_match = holder[0].iloc[1,:] == holder[1].iloc[1,:]
                ## Check if tables have matching headers for special 
                ## election years + storage
                if is_match.sum() == 3:
                    fin_tab = holder[1].append(holder[0], ignore_index=True)
                    yr_summaries[year] = fin_tab
            ## Storage of non-special election years
            else:
                try:
                    yr_summaries[year] = holder.pop()
                except:
                    continue
        
    return yr_summaries

In [7]:
def ref_tabler(summary_df, mod_df=True):
    ## Containers for mid-results
    st_list = []
    sntr_list = []
    
    ## Looping through states' names
    for state in summary_df['State']:
        ## Check/replace spaces with '_' + storage
        if ' ' in state:
            fmt_state = state.replace(' ', '_')
            st_list.append(fmt_state)
        else:
            st_list.append(state)
    ## Looping + storing senator names
    for name in summary_df['Senator']:
        sntr_list.append(name)

    ## Dataframe with Sentors as index + states as values
    state_ref_df = pd.DataFrame(st_list, index=sntr_list)
    state_ref_df.columns = ['State_id']
    
    ## Modifying list of states to that of the ref table OR not + return
    if mod_df:
        summary_df['State'] = st_list
        return summary_df, state_ref_df
    else:
        return state_ref_df

In [8]:
def yr_sum_formatter(yr_sum_dict, ref_table=True):
    ## Container for results + column helper
    res_dict = {}
    df_cols = ['State', 'Senator', 'Party', 'Electoral_History', 'Results', 'Candidates']
    
    ## Looping to each year's summary table
    for year in yr_sum_dict:
        ## Setting copy to modify + setting columns + rows to remove
        summary_df = yr_sum_dict[year].copy()
        summary_df.columns = df_cols
        drop_1 = summary_df.iloc[0]
        drop_2 = summary_df.iloc[1]
        
        ## Removing rows that match 'drop_' rows (former column info)
        for idx, row in summary_df.iterrows():
            if drop_1.equals(row):
                summary_df.drop(idx, inplace=True)
            elif drop_2.equals(row):
                summary_df.drop(idx, inplace=True)
        
        ## Resetting index to rangeIndex
        summary_df.reset_index(drop=True, inplace=True)
        
        ## Storing State-Senator lookup dataframe OR not
        if ref_table:
            summary_df, state_ref_df = ref_tabler(summary_df)
            res_dict[year] = [summary_df, state_ref_df]    
        else:
            res_dict[year] = summary_df
        
    ## Return results
    return res_dict

# Initial Data

> [Link to Dataset](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/PEJ5QU)

In [ ]:
df = pd.read_csv('dataverse_files/1976-2018-senate.csv', encoding='latin-1')
df.head()

In [ ]:
df.info()

In [ ]:
df[(df['state_po'] == 'AZ') & (df['year'] == 1976)]

In [ ]:
# len(df[df['candidate'].isna()])

# df['writein'].value_counts()

# for col in df:
#     display(df[col].value_counts())

# pd.Series.value_counts()

# Additional Data

Current set of features includes:
* Year of election
* State (encoded)
* 

> [Link to Brookings](https://www.brookings.edu/multi-chapter-report/vital-statistics-on-congress/)

> [Link to BallotPedia](https://ballotpedia.org/Legislative_Branch)

> [Link to WikiPedia](https://en.wikipedia.org/wiki/List_of_United_States_Senate_elections)

> [Link to Wiki-Category (U.S. Senator)](https://commons.wikimedia.org/wiki/Category:Senators_of_the_United_States)

## Testing tables 'infobox vevent'

In [ ]:
# url3 = 'https://en.wikipedia.org/wiki/1978_United_States_Senate_elections'
# response3 = requests.get(url3)
# print(response3)

# soup3 = BeautifulSoup(response3.text, 'html.parser')
# wiki_tables3 = soup3.findAll('table', attrs={'class': ['wikitable', 'infobox vevent']})

# print(type(wiki_tables3))

# test_df3 = pd.read_html(str(wiki_tables3))

# tester = test_df3[0]

In [ ]:
# https://stackoverflow.com/questions/27156278/index-pandas-dataframe-by-column-numbers-when-column-names-are-integers
# tester.columns = ['a', 'b', 'c']
# display(tester.iloc[6:18,0:3])

# display(test_df3[2])

In [ ]:
# url = 'https://en.wikipedia.org/wiki/1976_United_States_Senate_elections'
# response = requests.get(url)
# print(response)

# soup = BeautifulSoup(response.text, 'html.parser')
# wiki_tables = soup.findAll('table', attrs={'class': 'wikitable'})

# type(wiki_tables)

In [ ]:
# test_df = pd.read_html(str(wiki_tables))
# display(test_df[0])

# len(test_df)

## Pulling state election tables

In [ ]:
# ## Pulling Arizona gen election results
# tester = test_df[5].copy()
# print('O.G. Copy:')
# print('---'*20)
# display(tester)

# ## Setting columns manually + Q.C.
# tester.columns = ['Null', 'Party', 'Candidate', 'Votes', '%of_Vote']
# print('\nNew Column Headers:')
# print('---'*20)
# display(tester)

# ## Dropping irrelevant information + Q.C.
# tester.drop(index=[0, 6, 8], inplace=True)
# print('\nDropping Rows:')
# print('---'*20)
# display(tester)

# ## Flip and place Turnout as its own feature + Q.C.
# for idx, v, k in zip(tester.index, tester.Null, tester.Null.isna()):
#     if not k:
#         #print(idx, v)
#         tester[v] = tester.Party[idx]

# ## Removing the row with turnout info/column with nulls + Q.C.
# try:
#     tester.drop(columns='Null', index=7, inplace=True)
#     print('\nFinal Version:')
#     print('---'*20)
#     display(tester)
# except:
#     print('Something went wrong!')

In [ ]:
# type(tester['Votes'][5]) # str

In [ ]:
# tester2 = gen_election_cleaner(test_df[6], verbose=2)

In [ ]:
# test_df[-1]

In [ ]:
# url2 = 'https://en.wikipedia.org/wiki/List_of_United_States_Senate_elections'
# response2 = requests.get(url2)
# print(response2)

In [ ]:
# url2 = 'https://en.wikipedia.org/wiki/2018_United_States_Senate_elections'
# response2 = requests.get(url2)
# print(response2)

In [ ]:
# soup2 = BeautifulSoup(response2.text, 'html.parser')
# wiki_toc = soup2.find('div', attrs={'id':'toc'})
# wiki_toc = wiki_toc.findAll('a', href=is_state)
# display(wiki_toc[0:5])
# jiji = wiki_toc[0:5]

# jiji = [tag.get('href').replace('#', '') for tag in jiji]
# jiji = jiji
# jiji

## Working

In [ ]:
# test = wiki_links[78:134]
# beeb = test[0].get_text()
# geeb = 'https://en.wikipedia.org'
# geeb + beeb

In [ ]:
# display(elect_yr_dfs.keys())
# display(elect_yr_tocs.keys())
# display(len(elect_yr_dfs) == len(elect_yr_tocs))
# elect_yr_dfs.keys() != elect_yr_tocs.keys()
# list(elect_yr_dfs.keys())

In [ ]:
# elect_yr_dfs['1918']

In [ ]:
# elect_yr_tocs['1918']

In [ ]:
# year = '1924'

# for i, df in enumerate(elect_yr_dfs[year]):
#     print(i, df.shape)

# display(elect_yr_dfs[year][10])
# elect_yr_dfs[year][10].shape[1] is 6

In [ ]:
# koko = {v:elect_yr_dfs[v] for v in ['1918', '1924', '1976', '2016', '2018']}
# lplp = {v:elect_yr_tocs[v] for v in ['1918', '1924', '1976', '2016', '2018']}

In [ ]:
# year = '2016'

# for v in ftft[year]:
#     print(len(v))
    
# ftft[year][1]

In [ ]:
# yuyu = test_res[-1].append(test_res[-2], ignore_index=True)
# yuyu

In [ ]:
# yuyu = test_res[-1].iloc[1,:] == test_res[-2].iloc[1,:]
# yuyu.sum()

In [ ]:
# len(test_res)

## Getting it together

In [ ]:
# elect_yr_tables, elect_yr_tocs = wiki_senate_scraper()

# election_collection_raw = election_collector(elect_yr_tables, elect_yr_tocs)

In [ ]:
# for u in election_collection_raw[year][1][0].iloc[:,4]:
#     if isinstance(u, str) and 'elected' in u:
#         print('faslidfjasedfhaedfng')
        
# for u in election_collection_raw[year][1][1].iloc[:,4]:
#     if isinstance(u, str) and 'elected' in u:
#         print('faslidfjasedfhaedfng')

In [ ]:
# print(election_collection_raw[year][1][0].shape)
# print(election_collection_raw[year][1][1].shape)

In [ ]:
# test_list = [election_collection_raw[year][2][8], election_collection_raw[year][2][3]]
# for u in test_list:
    
#     display(u.iloc[:,2])

In [ ]:
# election_collection_raw[year][1][1]
# for i, df in enumerate(election_collection_raw[year][2]):
#     display(df)

## Loading `Election_Collection`

In [ ]:
# with open('election_collection_raw.pickle', 'wb') as f:
#     pickle.dump(election_collection_raw, f)
#     f.close()

In [10]:
with open('election_collection_raw.pickle', 'rb') as f:
    election_collection_raw = pickle.load(f)
    f.close()

## Separation/Cleaning

### Testing

In [ ]:
# test_res_dict = yr_sum_formatter(test_res)
# for table in test_res_dict['2008']:
#     display(table)

In [ ]:
# year = '2006'

# display(election_collection_raw.keys())
# print('\n')
# for grp in election_collection_raw[year]:
#     print(len(grp))
# print('\n')
# display(election_collection_raw[year][0])
# display(election_collection_raw[year][1])
# display(election_collection_raw[year][2])

In [ ]:
# test_list = []
# test_list.append(election_collection_raw['1918'])
# test_list.append(election_collection_raw['1920'])
# test_list.append(election_collection_raw['1976'])
# test_list.append(election_collection_raw['2006'])
# test_list.append(election_collection_raw['2008'])

# test_list = ['1918', '1920', '1976', '2006', '2008']
# test_dict = {}
# for year in test_list:
#     test_dict[year] = election_collection_raw[year]
# test_dict['2008']

In [ ]:
# type(election_collection_raw['1994'])

In [ ]:
# print(type(election_collection_raw['1920']))
# print(type(election_collection_raw['1976']))
# type(test_res['1920'])

In [ ]:
# test_res = yr_summary_collector(test_dict)
# for t in test_res:
#     display(test_res[t])

### Working..

**To format the tables I need to:**
* For each table in the dictionary, set the columns to `'State', 'Senator', 'Party', 'Electoral_History', 'Results', 'Candidates'`.
* If any of the rows look like row 0 & 1, remove them.
* Store the formatted tables back into container

In [ ]:
yr_sum_tables = yr_summary_collector(election_collection_raw)
yr_sum_dict = yr_sum_formatter(yr_sum_tables)

In [15]:
for table in yr_sum_dict['1994']:
    display(table)

,State,Senator,Party,Electoral_History,Results,Candidates
0,Arizona,Dennis DeConcini,Democratic,197619821988,Incumbent retired.New senator elected.Republic...,Jon Kyl (Republican) 53.7% Sam Coppersmith (De...
1,California,Dianne Feinstein,Democratic,1992 (Special),Incumbent re-elected.,Dianne Feinstein (Democratic) 46.7% Michael Hu...
2,Connecticut,Joe Lieberman,Democratic,1988,Incumbent re-elected.,Joe Lieberman (Democratic) 67% Jerry Labriola ...
3,Delaware,William Roth,Republican,19701971 (Appointed)197619821988,Incumbent re-elected.,William Roth (Republican) 55.8% Charles Oberly...
4,Florida,Connie Mack III,Republican,1988,Incumbent re-elected.,Connie Mack III (Republican) 70.5% Hugh Rodham...
5,Hawaii,Daniel Akaka,Democratic,1990 (Appointed)1990 (Special),Incumbent re-elected.,Daniel Akaka (Democratic) 71.8% Maria Hustace ...
6,Indiana,Richard Lugar,Republican,197619821988,Incumbent re-elected.,Richard Lugar (Republican) 67.4% Jim Jontz (De...
7,Maine,George J. Mitchell,Democratic,1980 (Appointed)19821988,Incumbent retired.New senator elected.Republic...,Olympia Snowe (Republican) 60.2% Thomas Andrew...
8,Maryland,Paul Sarbanes,Democratic,197619821988,Incumbent re-elected.,Paul Sarbanes (Democratic) 59.1% Bill Brock (R...
9,Massachusetts,Ted Kennedy,Democratic,1962 (Special)19641970197619821988,Incumbent re-elected.,Ted Kennedy (Democratic) 58.1% Mitt Romney (Re...


,State_id
Dennis DeConcini,Arizona
Dianne Feinstein,California
Joe Lieberman,Connecticut
William Roth,Delaware
Connie Mack III,Florida
Daniel Akaka,Hawaii
Richard Lugar,Indiana
George J. Mitchell,Maine
Paul Sarbanes,Maryland
Ted Kennedy,Massachusetts


* Create a table with names as index and state as values .loc name of candidates to get state value
* Use table of each year's general summary to then pull the general elections out of whole list (index 2 in colleciton) based upon names in table + non-uniform party column
    * Need to merge (if special election) tables in collection index 1

In [ ]:
test = {'1918': ['fire'], '1920': 'jet', '1976': 'hello'}
test['1918'].append('bromine')
for i in test:
    print(i)
print(test['1918'])

* 1924 = 10
* 1926 = 12
* 1928 = 10
* Pull only tables with `shape[1]` of 5 or 6.
* Table 2 is always summary of senate leaders

Why after the 17th amendment?? [Link](https://en.wikipedia.org/wiki/Seventeenth_Amendment_to_the_United_States_Constitution)

For 1920 forward, proceed like 1976. Prior to this, will need to use first table with general info.

* **Clean up NaN value table from top of pages** -- Use index 2 to grab  nested table from `'infobox vevent'`.
* **Take list of states of top tables** and map to dfs
* Get demographic info wikidata
* eda time series of state party (heatmap)
* NLP on names, etc.
* [link for slider viz](https://medium.com/@sjacks/the-journey-to-an-attractive-visualization-bac019506a49)